In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
import random
import datetime

In [3]:
w_length = 300

In [4]:
def splitSeconds(n, country, t):
    print("getting file")
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=True)
    samples = []
    for track in tracks:
        try:
            print("getting feats")
            trackFeats = data[data.track_id == track]
            FeatsLen =  trackFeats.shape[0]
            ind = random.randrange(1, FeatsLen - 10)
            feats = trackFeats.iloc[ind:ind+(w_length*10),6:30]
            dur = trackFeats.iloc[ind:ind+(w_length*10),1]
            example = np.array(feats.loc[feats.index.repeat(dur * 100)][-w_length:])
            if example.shape[0] == w_length:
                samples = samples + [example]
        except:
            continue
    samples = np.array(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [97]:
def getSamples(train_n, val_n):
    train = pd.DataFrame()
    train_labels = pd.DataFrame()
    val = pd.DataFrame()
    val_labels = pd.DataFrame()
    train_x = []
    train_labels = []
    val_x = []
    val_labels = []
    for country in countriesOfInterest:
        print(country)
        x1, y1 = splitSeconds(train_n, country, "train")
        x2, y2 = splitSeconds(val_n, country, "val")
        print("collected")
        train_x = train_x + x1.tolist()
        train_labels = train_labels + y1.tolist()
        val_x = val_x + x2.tolist()
        val_labels = val_labels + y2.tolist()
    print("cleaning")
    train_x = np.array(train_x)
    print(train_x.shape)
    train_labels = np.array(train_labels)
    val_x = np.array(val_x)
    val_labels = np.array(val_labels)
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(train_labels),
                                                     list(train_labels))
    #train_labels = enc.transform(np.array(train_labels).reshape(-1,1)).toarray()
    #val_labels = enc.transform(np.array(val_labels).reshape(-1,1)).toarray()
    return train_x, train_labels, val_x, val_labels, class_weights

In [ ]:
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]

splitSeconds(500, "ZA", "train")
#getSamples(50,10)

getting file
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting feats
getting

In [87]:
country = "ZA"
t = "train"
n = 50
print("getting file")
data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
tracks = data.track_id.unique()
tracks = np.random.choice(tracks, size=n, replace=False)
samples = []

getting file


In [91]:
def splitSeconds(n, country, t, seconds, samplerate):
    length = seconds * samplerate
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=False)
    trackFeats = data[data.track_id.isin(tracks)]
    dur = trackFeats.iloc[:,1]
    long = trackFeats.loc[trackFeats.index.repeat(dur * samplerate)].reset_index(drop = True)
    long['change'] = long.track_id.eq(long.track_id.shift())
    change = long[long.change == False].index
    long = long.iloc[:, 5:30]
    indices = np.concatenate((np.arange(0, long.shape[0], length), change))
    indices = np.sort(indices)
    indices = np.unique(indices)
    partition = np.split(np.array(long), indices)
    samples = []
    for i in partition:
        if i.shape[0] == length:
            samples = samples + [i]
    samples = np.stack(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [90]:
get

array(['ZA', 'ZA', 'ZA', ..., 'ZA', 'ZA', 'ZA'], dtype='<U2')

In [98]:
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]

train_x, train_labels, val_x, val_labels, class_weights = getSamples(200, 50)

ZA
getting file
repeating
partitioning
checking
getting file
repeating
partitioning
checking
collected
EG
getting file
repeating


KeyboardInterrupt: 

In [96]:
class_weights

array([1.11811061, 1.11369618, 0.96462303, 1.06331897, 0.85851449,
       0.95114804, 0.8683304 , 1.22721152, 1.29650908, 0.73816199,
       1.1909429 , 0.92399782])